In [1]:
# variables
DATA_PATH = 'data/raw/'

CONVERTED_DATA_PATH = 'data/magfit/'
MAGFIT_RES_PATH = 'results/magfit/'
MAGGEN_RES_PATH = 'results/maggen/'

MAG_RES_PATH = 'results/mag/'

In [2]:
import networkx as nx
import numpy as np
from scipy.sparse import csr_matrix
import utils.project_utils as p_utils
import os
import subprocess

In [3]:
#datasets = ['polblogs', 'citeseer', 'cora', 'cora_ml', 'pubmed']
datasets = ['polblogs']

In [4]:
# prepare data for MAGFIT
for dataset in datasets:
    path = f'{DATA_PATH}/{dataset}.npz'
    data = p_utils.open_npz(path)

    matrix = csr_matrix((data['adj_data'], data['adj_indices'], data['adj_indptr']))
    shape = matrix.shape

    while shape[0] > shape[1]:
        matrix._shape = (shape[0], shape[1]+1)
        matrix.indptr = np.hstack((matrix.indptr,matrix.indptr[-1]))
        shape = matrix.shape

    G = nx.convert_matrix.from_scipy_sparse_matrix(matrix)
    n_of_attrs = data['attr_shape'][1]

    p_utils.prepare_magfit_input_data(dataset, CONVERTED_DATA_PATH, G, n_of_attrs)
    

Creation of the directory data/magfit/polblogs/ failed


In [5]:
# MAGFIT command generation
for dataset in datasets:
    input = os.path.abspath(f'{CONVERTED_DATA_PATH}{dataset}/graph.txt').replace(' ', '\ ')
    config = os.path.abspath(f'{CONVERTED_DATA_PATH}{dataset}/inti.config').replace(' ', '\ ')
    output = p_utils.prepare_mag_result_file(MAGFIT_RES_PATH, dataset)
    output = os.path.abspath(output).replace(' ', '\ ')

    command = f'mag_algs/magfit -i:{input} -pi:{config} -o:{output} -fe:F -fm:F'
    print(command)
    print('\n\n')

Creation of the directory results/magfit/polblogs/ failed
mag_algs/magfit -i:/Users/andrejantonov/Google\ Диск/Учеба/НИР,\ Антонов\ Андрей,\ J4133c/Experiments/data/magfit/polblogs/graph.txt -pi:/Users/andrejantonov/Google\ Диск/Учеба/НИР,\ Антонов\ Андрей,\ J4133c/Experiments/data/magfit/polblogs/inti.config -o:/Users/andrejantonov/Google\ Диск/Учеба/НИР,\ Антонов\ Андрей,\ J4133c/Experiments/results/magfit/polblogs/polblogs.txt -fe:F -fm:F





In [6]:
# MAGGEN command generation
for dataset in datasets:
    n_of_nodes = 3000
    output = p_utils.prepare_mag_result_file(MAGGEN_RES_PATH, dataset)
    output = os.path.abspath(output).replace(' ', '\ ')
    simple = False
    conf_file = os.path.abspath(f'{CONVERTED_DATA_PATH}{dataset}/inti.config').replace(' ', '\ ')

    command = f'mag_algs/maggen -i:{conf_file} -o:{output} -n:{n_of_nodes} -simple:"F"' if not simple else f'mag_algs/maggen -o:{output} -n:{n_of_nodes}'
    print(command)
    print('\n\n')


Successfully created the directory results/maggen/polblogs/ 
mag_algs/maggen -i:/Users/andrejantonov/Google\ Диск/Учеба/НИР,\ Антонов\ Андрей,\ J4133c/Experiments/data/magfit/polblogs/inti.config -o:/Users/andrejantonov/Google\ Диск/Учеба/НИР,\ Антонов\ Андрей,\ J4133c/Experiments/results/maggen/polblogs/polblogs.txt -n:3000 -simple:"F"





In [8]:
# Converting result files to networkx format

for dataset in ['polblogs']:
    structure_file = p_utils.get_mag_result_file(MAGGEN_RES_PATH, dataset)
    features_file = p_utils.get_mag_result_file(MAGFIT_RES_PATH, dataset) + 'f'

    struct = open(structure_file, 'r')
    features = open(features_file, 'r')

    p_utils.save_mag_fetures(features, f'{MAG_RES_PATH}{dataset}/', 'Features.csv')

    struct_arr = []

    G = nx.Graph()

    for row in struct:
        if row[0] == '#': continue
        nodes = row.split('\t')
        G.add_edge(int(nodes[0]), int(nodes[1][:len(nodes[1])-1]))
    
    p_utils.save_graph(G, f'{MAG_RES_PATH}{dataset}/', '/G.csv')

Successfully created the directory results/mag/polblogs/ 
Creation of the directory results/mag/polblogs/ failed
